In [ ]:
import requests
import json
import pandas as pd

In [ ]:
query_1='''
"graph database" OR "graph databases" OR Neo4j OR "Azure Cosmos" OR ArangoDB OR OrientDB OR (Openlink AND Virtuoso)
OR (Ontotext AND GraphDB)  OR JanusGraph  OR "Amazon Neptune" OR (Stardog AND "graph database") OR TigerGraph
OR FaunaDB OR (Fauna AND "graph database") OR AllegroGraph  OR (Dgraph AND "graph database")
OR (Giraph AND "graph database") OR "Nebula Graph"
'''

query_1='+'.join(query_1.split(' '))

In [ ]:
query_2='''
Neo4j OR "Azure Cosmos" OR ArangoDB OR OrientDB OR (Openlink AND Virtuoso) OR (Ontotext AND GraphDB) OR JanusGraph
OR "Amazon Neptune" OR (Stardog AND "graph database") OR TigerGraph OR FaunaDB OR (Fauna AND "graph database")
OR AllegroGraph  OR (Dgraph AND "graph database") OR (Giraph AND "graph database") OR "Nebula Graph"

'''

query_2='+'.join(query_2.split(' '))

In [ ]:
pmd_id_request='https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&retmax=100000&term='


In [ ]:
pmc_id_request='https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&retmode=json&retmax=100000&term='


In [ ]:
pmd_id_request+=query_1
pmd_id_result=requests.get(pmd_id_request)
pmd_id_result_json=json.loads(pmd_id_result.text)

In [ ]:
pmc_id_request+=query_2
pmc_id_result=requests.get(pmc_id_request)
pmc_id_result_json=json.loads(pmc_id_result.text)

In [ ]:
pmd_id_list=pmd_id_result_json['esearchresult']['idlist']

In [ ]:
pmc_id_list=pmc_id_result_json['esearchresult']['idlist']

In [ ]:
pmd_data_list=[]
iteration=100

for i in range(len(pmd_id_list)//iteration):
    pmd_data_request='https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&retmode=json&id='
    
    pmd_ids=','.join(pmd_id_list[i*iteration:(i+1)*iteration])
    pmd_data_request+=pmd_ids
    pmd_data_result=requests.get(pmd_data_request)
    pmd_data_result_json=json.loads(pmd_data_result.text)
    
    for j in list(pmd_data_result_json['result'].keys())[1:]:
    
        temp_dict=pmd_data_result_json['result'][j]
        
        if 'articleids' in temp_dict.keys():
            for k in temp_dict['articleids']:

                temp_articleid= k['idtype']
                temp_articleid_value= k['value']

                temp_dict[temp_articleid]=temp_articleid_value

            pmd_data_list.append(temp_dict)
            
pmd_data_request='https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&retmode=json&id='            
    
pmd_ids=','.join(pmd_id_list[(len(pmd_id_list)//iteration)*iteration:(len(pmd_id_list)//iteration)*iteration+(len(pmd_id_list)%iteration)])
pmd_data_request+=pmd_ids
pmd_data_result=requests.get(pmd_data_request)
pmd_data_result_json=json.loads(pmd_data_result.text)

for j in list(pmd_data_result_json['result'].keys())[1:]:

    temp_dict=pmd_data_result_json['result'][j]

    if 'articleids' in temp_dict.keys():
        for k in temp_dict['articleids']:

            temp_articleid= k['idtype']
            temp_articleid_value= k['value']

            temp_dict[temp_articleid]=temp_articleid_value

        pmd_data_list.append(temp_dict)

In [ ]:
pmc_data_list=[]
iteration=400

for i in range(len(pmc_id_list)//iteration):
    pmc_data_request='https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pmc&retmode=json&id='
    
    pmc_ids=','.join(pmc_id_list[i*iteration:(i+1)*iteration])
    pmc_data_request+=pmc_ids
    pmc_data_result=requests.get(pmc_data_request)
    pmc_data_result_json=json.loads(pmc_data_result.text)
    
    for j in list(pmc_data_result_json['result'].keys())[1:]:
    
        temp_dict=pmc_data_result_json['result'][j]
        
        if 'articleids' in temp_dict.keys():
            for k in temp_dict['articleids']:

                temp_articleid= k['idtype']
                temp_articleid_value= k['value']

                temp_dict[temp_articleid]=temp_articleid_value

            pmc_data_list.append(temp_dict)
            
pmc_data_request='https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pmc&retmode=json&id='            
    
pmc_ids=','.join(pmc_id_list[(len(pmc_id_list)//iteration)*iteration:(len(pmc_id_list)//iteration)*iteration+(len(pmc_id_list)%iteration)])
pmc_data_request+=pmc_ids
pmc_data_result=requests.get(pmc_data_request)
pmc_data_result_json=json.loads(pmc_data_result.text)

for j in list(pmc_data_result_json['result'].keys())[1:]:

    temp_dict=pmc_data_result_json['result'][j]

    if 'articleids' in temp_dict.keys():
        for k in temp_dict['articleids']:

            temp_articleid= k['idtype']
            temp_articleid_value= k['value']

            temp_dict[temp_articleid]=temp_articleid_value

        pmc_data_list.append(temp_dict)

In [ ]:
pmd_df=pd.DataFrame(pmd_data_list)

In [ ]:
pmc_df=pd.DataFrame(pmc_data_list)

In [ ]:
both_df=pmd_df.append(pmc_df)

In [ ]:
both_df=both_df.fillna('')

In [ ]:
both_df=both_df.drop_duplicates('title')
both_df=both_df.reset_index(drop=True)

In [ ]:
both_df=both_df[['pubdate', 'authors', 'title', 'fulljournalname', 'pubmed', 'doi', 'pmc']]

In [ ]:
both_df.columns=['Date', 'Author', 'Title', 'Publication Title', 'PMID', 'DOI', 'PMCID']

In [ ]:
for i in range(len(both_df)):
    if both_df.iloc[i]['PMCID']!='':
        both_df.at[i, 'Link Attachments']='https://www.ncbi.nlm.nih.gov/pmc/articles/'+both_df.iloc[i]['PMCID']
    elif both_df.iloc[i]['PMID']!='':
        both_df.at[i, 'Link Attachments']='https://pubmed.ncbi.nlm.nih.gov/'+both_df.iloc[i]['PMID']
    else:
        both_df.at[i, 'Link Attachments']='https://doi.org/'+both_df.iloc[i]['DOI']
        
    names=[]
    for j in both_df.iloc[i]['Author']:
        names.append(j['name'])
    both_df.at[i, 'Author']=', '.join(names)